# Data Paths

In [1]:
# MICC_F2000
F2000_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/F2000/'
# MICC_F220
F220_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/F220/'
# MICC_F600
F600_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/F600/'
# CoMoFoD
CoMoFoD_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/CoMoFoD/'
#CMFDdb_grip
GRIP_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/GRIP/'
#CMFD Dataset 0
CVIP0_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/CVIP0/'
#CMFD Dataset 1-2
CVIP12_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/CVIP12/'
#FFCMF
FFCMF_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/FFCMF/'
#COVERAGE
COVERAGE_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/COVERAGE/'
# CASIA_PATH
CASIA_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/CASIA/'
# COCO
COCO_DATA_PATH='/media/ansary/DriveData/RESEARCH/CMFD/DATA/coco/train2014/'
annFile='/media/ansary/DriveData/RESEARCH/CMFD/DATA/coco/annotations/instances_train2014.json'
# Common Params
SAVE_PATH='/media/ansary/DriveData/RESEARCH/CMFD/'
IMG_DIM=256


# Image And Target Creation

In [2]:
import os
from glob import glob
from dataset import F2000,F220,F600,CoMoFoD,GRIP,CVIP0,CVIP12,FFCMF,COVERAGE,CASIA
from tqdm.notebook import tqdm
# datasets
DS=[CoMoFoD,CASIA,COVERAGE,CVIP12,FFCMF,CVIP0,GRIP,F600,F220,F2000]
PATHS=[CoMoFoD_PATH,CASIA_PATH,COVERAGE_PATH,CVIP12_PATH,FFCMF_PATH,CVIP0_PATH,GRIP_PATH,F600_PATH,F220_PATH,F2000_PATH]
DS_DIR=os.path.join(SAVE_PATH,'DataSet')
TRAIN_DIR=os.path.join(DS_DIR,'Train')
EVAL_DIR=os.path.join(DS_DIR,'Eval')

for dpath,obj in zip(PATHS,DS):
    ds=obj(dpath,SAVE_PATH,IMG_DIM)
    ds.prepare()





Initializing:CVIP12



Initializing:FFCMF



Initializing:CVIP0



Initializing:GRIP



Initializing:F600



Initializing:F220



Initializing:F2000


# COCO for creating synthetic data

In [3]:
from pycocotools.coco import COCO
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import random
import collections
from tqdm.notebook import tqdm

In [4]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=11.54s)
creating index...
index created!


In [5]:
_img=os.path.join(TRAIN_DIR,'images')
_msk=os.path.join(TRAIN_DIR,'masks')
catIds = coco.getCatIds()
imgIds = coco.getImgIds()
file_name_list=[((coco.loadImgs(imid)[0])['file_name']) for imid in imgIds]

def findAnnotationMatchingCriteria(anns,imgdata,lowerCutoff=0.01,upperCutoff=0.2):
    annotation_count=len(anns)    
    toReturn=[]
    indexes=random.sample(range(annotation_count),annotation_count)
    while len(indexes)>0:
        h,w,c=imgdata.shape
        mask=coco.annToMask(anns[indexes.pop()])
        forgedPixelsCount=collections.Counter(mask.flatten())[1]  
        if forgedPixelsCount>= (h*w*lowerCutoff) and forgedPixelsCount<=(h*w*upperCutoff):
            toReturn=mask
            break
        
    return toReturn



def getAffineTransformedMask(foreground,binarymask):
    
    indices = np.where(binarymask == 1)
    upper = np.min(indices[0])
    lower = np.max(indices[0])
    left = np.min(indices[1])
    right = np.max(indices[1])
    
    width = right - left
    height= lower - upper
    n = random.randint(10,100)
    hor_right = False if (binarymask.shape[1] - (right+ n +width)) <= 0 else True
    hor_left = False if (left - (n+width)) <= 0 else True 

    side = ""
    if hor_right == True and hor_left == True:
        side=random.sample(["R","L"],1)[0]
        
    elif hor_right == True and hor_left == False:
        side = "R"

    elif hor_right == False and hor_left == True:
        side = "L"
    else:
        return([],[])

    if side == "L":
        S = -(width+n)
    else:
        S = width+n

    v = 0    
    lu = random.randint(0,1)
    if((upper-10)>1 and (binarymask.shape[0]-lower-10) >1):
        if lu == 1:
            v = -random.randint(1,upper-10)
        else:
            v = random.randint(1,binarymask.shape[0]-lower-10)
    elif (upper-10)>1 :
        v = -random.randint(1,upper-10)
    elif (binarymask.shape[0]-lower-10) >1:
        v = random.randint(1,binarymask.shape[0]-lower-10)
    else:
        return([],[])
        
    rows,cols = binarymask.shape
    
    M = np.float32([[1,0,S],[0,1,v]])
    transformedForeground = cv2.warpAffine(foreground,M,(cols,rows))
    transformedBinaryMask= cv2.warpAffine(binarymask,M,(cols,rows))
    return(transformedForeground,transformedBinaryMask)
        
def transform(binarymask,imgdata,double_forge=False):
    if double_forge:
        scale=1
        angle=0
    else:
        scale=round(random.uniform(0.2,1.8),1) 
        angle=random.randint(0,90)

    imgdata=Image.fromarray(imgdata)
    imgdata=imgdata.resize((int(imgdata.size[0]*scale),int(imgdata.size[1]*scale)))
    imgdata=imgdata.rotate(angle)
    foreground=np.array(imgdata).copy()
    imgdata=np.array(imgdata)
    
    binarymask=Image.fromarray(binarymask)
    binarymask=binarymask.resize((int(binarymask.size[0]*scale),int(binarymask.size[1]*scale)))
    binarymask=binarymask.rotate(angle,Image.NEAREST)
    binarymask=np.array(binarymask)
    
    
    
    foreground[:,:,0]=np.array(imgdata[:,:,0] * binarymask )
    foreground[:,:,1]=np.array(imgdata[:,:,1] * binarymask )
    foreground[:,:,2]=np.array(imgdata[:,:,2] * binarymask )
    
    
    return foreground,binarymask
    

In [7]:
counter=len([img_path for img_path in glob(os.path.join(TRAIN_DIR,'images','*.*'))])


src = list(zip(imgIds,file_name_list))

random.shuffle(src)

imgIds,file_name_list= zip(*src)

for imgId,file_name in tqdm(zip(imgIds,file_name_list),total=len(file_name_list)):
    
        imgdata=cv2.imread(os.path.join(COCO_DATA_PATH,file_name))
        imgdata=cv2.cvtColor(imgdata,cv2.COLOR_BGR2RGB)
        
        annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        binarymask=findAnnotationMatchingCriteria(anns,imgdata)
        if len(binarymask)==0:
            continue
        # originals    
        source=binarymask.copy()
        background=Image.fromarray(imgdata,'RGB')
        # nb forge
        nb_forge=random.choices(population=[1,2],weights=[0.6,0.4],k=1)[0]
        if nb_forge==2:
            double_forge=True
        else:
            double_forge=False
          
       
        #transformation
        foreground,binarymask=transform(binarymask,imgdata,double_forge)
         # zero area exception
        try:
            indices = np.where(binarymask == 1)
            upper = np.min(indices[0])
            lower = np.max(indices[0])
            left = np.min(indices[1])
            right = np.max(indices[1])
        except:
            continue
            
        forges=[]
        for i in range(nb_forge):
            new_foreground,new_binarymask=getAffineTransformedMask(foreground,binarymask)
            if(len(new_foreground)==0):
                continue
            new_foreground=Image.fromarray(new_foreground,'RGB').convert('RGBA')
            new_binarymask=Image.fromarray(new_binarymask*255)
            #image
            datas=new_foreground.getdata()
            newData = []
            for item in datas:
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append((0, 0, 0, 0))
                else:
                    newData.append(item)
            new_foreground.putdata(newData)

            background.paste(new_foreground,(0,0),mask=new_foreground.split()[3])
            # mask
            datas=new_binarymask.getdata()
            newData = []
            for item in datas:
                if item==0:
                    newData.append(0)
                else:
                    newData.append(item)
            forge=Image.fromarray(np.zeros(source.shape))
            forge.paste(new_binarymask,(0,0))
            forge=np.array(forge)
            forge=forge.astype('uint8')
            forges.append(forge)
            
        
        if len(forges)==0:
            continue
        source=(source*255).astype('uint8')
        
        for forge in forges:
            source=source+forge
            source[source>0]=255
            
        source= cv2.resize(source,(IMG_DIM,IMG_DIM),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
        background=background.resize(size=(IMG_DIM,IMG_DIM))
        background.save(os.path.join(_img,'{}.png'.format(counter)))
        cv2.imwrite(os.path.join(_msk,'{}.png'.format(counter)),source)
        counter+=1
        

# TFRecords For GCS

In [8]:
import tensorflow as tf 
import imghdr
tf.__version__

import math

# record dir
def create_dir(_path):
    if not os.path.exists(_path):
        os.mkdir(_path)
    return _path

tf_dir=create_dir(os.path.join(DS_DIR,'CMFDTF'))
tf_train=create_dir(os.path.join(tf_dir,'Train'))
tf_eval=create_dir(os.path.join(tf_dir,'Eval'))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
      return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _float_feature(value):
      return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

data_num    = 256

def to_tfrecord(image_paths,save_dir,r_num):
    tfrecord_name='{}.tfrecord'.format(r_num)
    tfrecord_path=os.path.join(save_dir,tfrecord_name)
    with tf.io.TFRecordWriter(tfrecord_path) as writer:    
        for image_path in image_paths:
            target_path=str(image_path).replace('images','masks')
            
            with(open(image_path,'rb')) as fid:
                image_png_bytes=fid.read()
            with(open(target_path,'rb')) as fid:
                target_png_bytes=fid.read()
            
            data ={ 'image':_bytes_feature(image_png_bytes),
                    'mask':_bytes_feature(target_png_bytes)
            }
            features=tf.train.Features(feature=data)
            example= tf.train.Example(features=features)
            serialized=example.SerializeToString()
            writer.write(serialized)


def genTFRecords(data_paths,mode_dir):
    _paths=[]
    for img_path in tqdm(data_paths):
        msk_path=str(img_path).replace('images','masks')
        if imghdr.what(img_path) is 'png' and imghdr.what(msk_path) is 'png':
            _paths.append(img_path)
    print('Found Valid Images:',len(_paths))
    random.shuffle(_paths)
    for i in tqdm(range(0,len(_paths),data_num)):
        image_paths= _paths[i:i+data_num]
        random.shuffle(image_paths)        
        r_num=i // data_num
        to_tfrecord(image_paths,mode_dir,r_num)

# TFRecord Generation

In [9]:
        
train_paths=[img_path for img_path in tqdm(glob(os.path.join(TRAIN_DIR,'images','*.*')))]
eval_paths=[img_path for img_path in tqdm(glob(os.path.join(EVAL_DIR,'images','*.*')))]

random.shuffle(train_paths)
random.shuffle(eval_paths)


genTFRecords(train_paths,tf_train)
genTFRecords(eval_paths,tf_eval)



Found Valid Images: 48105



Found Valid Images: 6311
